In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms

import matplotlib.pyplot as plt
import csv

import cv2

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import shutil
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **PREPROCESSING**

In [ ]:
### PREPROCESS IN PYTORCH

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.3418, 0.3126, 0.3224], std=[0.1627, 0.1632, 0.1731])
])

# DATA LOADING

In [ ]:
trainset = torchvision.datasets.ImageFolder("../input/trafficsign-training-data/Final_Training/Images", transform = preprocess)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, drop_last=False)

In [ ]:
testset = torchvision.datasets.ImageFolder("../input/tagged-test-images-20/new_test_imgs", transform = preprocess)
test_loader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True, drop_last=True)

In [ ]:
class BigNet(nn.Module):
    def __init__(self):
        super().__init__()
    
        self.conv1 = nn.Conv2d(3, 32, 3, 1,1)
        self.conv_bn_1 = nn.BatchNorm2d(32)
        torch.nn.init.xavier_normal_(self.conv1.weight)
        torch.nn.init.zeros_(self.conv1.bias)
        
        self.conv2 = nn.Conv2d(32, 64, 3, 1,1)
        self.conv_bn_2 = nn.BatchNorm2d(64)
        torch.nn.init.xavier_normal_(self.conv2.weight)
        torch.nn.init.zeros_(self.conv2.bias)
        
        self.conv3 = nn.Conv2d(64, 128, 3,1,1)
        self.conv_bn_3 = nn.BatchNorm2d(128)
        torch.nn.init.xavier_normal_(self.conv3.weight)
        torch.nn.init.zeros_(self.conv3.bias)
        
        self.conv4 = nn.Conv2d(128, 256 , 3, 1,1)
        self.conv_bn_4 = nn.BatchNorm2d(256)
        torch.nn.init.xavier_normal_(self.conv4.weight)
        torch.nn.init.zeros_(self.conv4.bias)
        
        
        self.conv5 = nn.Conv2d(256, 512 , 3, 1,1)
        self.conv_bn_5 = nn.BatchNorm2d(512)
        torch.nn.init.xavier_normal_(self.conv5.weight)
        torch.nn.init.zeros_(self.conv5.bias)
        
        self.conv6 = nn.Conv2d(512, 1024 , 3, 1,1)
        self.conv_bn_6 = nn.BatchNorm2d(1024)
        torch.nn.init.xavier_normal_(self.conv6.weight)
        torch.nn.init.zeros_(self.conv6.bias)



        self.pool  = nn.MaxPool2d(2,2)

        self.act   = nn.ReLU(inplace=False)
        self.drop = nn.Dropout2d(0.2)    

        self.mlp = nn.Sequential(
            nn.Linear(4 * 4 * 1024, 128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, 43)     
        )
 
    def forward(self, x):
        x = self.conv_bn_1(self.conv1(x))
        x = self.pool(self.act(x))

        x = self.conv_bn_2(self.conv2(x))
        x = self.pool(self.act(x))

        x = self.conv_bn_3(self.conv3(x))
        x = self.pool(self.act(x))

        x = self.conv_bn_4(self.conv4(x))
        x = self.pool(self.act(x))

        x = self.conv_bn_5(self.conv5(x))
        x = self.pool(self.act(x))
        
        x = self.conv_bn_6(self.conv6(x))
        x = self.pool(self.act(x))
        

        bsz, nch, height, width = x.shape
        x = torch.flatten(x, start_dim=1, end_dim=-1)
        
        y = self.mlp(x)

        return y

In [ ]:
def model_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [ ]:
device = "cuda:0"

In [ ]:
cnn = BigNet()
print(model_params(cnn))
cnn = cnn.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn.parameters(), lr=0.01, momentum = 0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.1)
epochs = 10
steps = 0
running_loss = 0
train_losses = []
test_losses = []
print_every = 100

In [ ]:
for epoch in range(epochs):
    steps = 0
    for inputs, labels in train_loader:
        steps += 1
        inputs, labels = inputs.to(device), labels.to(device)
       
        optimizer.zero_grad()
        
        logps = cnn.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy_t = 0
            cnn.eval()
            with torch.no_grad():
                for inputs_val, labels_val in test_loader:
                    inputs_val, labels_val = inputs_val.to(device), labels_val.to(device)
                    
                    logps_t = cnn.forward(inputs_val)
                    batch_loss = criterion(logps_t, labels_val)
                    test_loss += batch_loss.item()
                    
                    top_p, top_class = logps_t.topk(1, dim=1)
                    equals = top_class == labels_val.view(*top_class.shape)
                    accuracy_t += torch.mean(equals.type(torch.FloatTensor)).item()
            
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(test_loader):.3f}.. "
                  f"Test accuracy: {accuracy_t/len(test_loader):.3f}")
            running_loss = 0
            cnn.train()
    scheduler.step()
    test_losses.append(test_loss/len(test_loader))
    train_losses.append(running_loss/len(train_loader))

In [ ]:
trial = torch.FloatTensor(1,3,256,256).to(device)

In [ ]:
path = "cnn.onnx"
path_pt = "cnn.pt"
torch.save(cnn.state_dict(), path_pt)
torch.onnx.export(cnn, trial, path)